<a href="https://colab.research.google.com/github/Anushkaghei/Underwater-Image-dehazing/blob/main/Evaluation_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

# RGB GRAPHS

RGB plots visualize the distribution of pixel intensities in the red, green, and blue channels of an image. They provide insights into the color composition and balance of an image. Often used for qualitative assessment of color distribution and balance in images. They can reveal issues such as color casts or saturation.

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image

# Open the original image
original1 = Image.open('/content/160_im.png') #path for raw image

gan1 = Image.open('/content/gen_1.png')# path for generated image

# Open the reference image
ref1 = Image.open('/content/160_img_.png')

# Resize the images to 256x256 pixels
original1 = original1.resize((256, 256))
gan1 = gan1.resize((256, 256))
ref1 = ref1.resize((256, 256))

def plot_histogram(image, text):
    # Convert image to RGB mode
    image = image.convert('RGB')
    # Split the R, G and B channels
    imageR, imageG, imageB = image.split()
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    plt.title(text)
    plt.imshow(image)
    plt.subplot(1, 2, 2)
    plt.title("Histogram of image")
    plt.plot(imageR.histogram(), color='red')
    plt.plot(imageG.histogram(), color='green')
    plt.plot(imageB.histogram(), color='blue')
    plt.show()

plot_histogram(gan1, "Generated Image")# replace title
plot_histogram(ref1, "Reference Image")


# PSNR & MSE (Peak Signal-to-Noise Ratio and Mean Squared Error):

PSNR measures the quality of an image by comparing it with a reference image. It quantifies the ratio between the maximum possible power of a signal and the power of corrupting noise that affects the fidelity of its representation. MSE measures the average squared difference between the pixels of the two images.
Higher PSNR values indicate higher image fidelity, while lower MSE values indicate better similarity between images.

In [ ]:
def psnr(reference, fused):
    # Convert images to RGB mode
    reference = reference.convert('RGB')
    fused = fused.convert('RGB')

    # Convert images to NumPy arrays
    reference_array = np.array(reference)
    fused_array = np.array(fused)

    # Calculate PSNR
    R2 = np.amax(reference_array)**2
    MSE = np.sum(np.power(np.subtract(reference_array, fused_array), 2))
    MSE /= (reference_array.shape[0] * reference_array.shape[1])
    PSNR = 10 * np.log10(R2 / MSE)

    print("Reference vs Generated   -", "MSE: ", MSE, "PSNR:", PSNR)
    print('')

print("MSE & PSNR of 160_img_.png")
psnr(ref1, gan1)


# SSIM (Structural Similarity Index):

SSIM compares the structural similarity between two images. It considers luminance, contrast, and structure similarity to provide a comprehensive measure of image quality. It is widely used for assessing the perceived quality of images. It is particularly effective in evaluating images subjected to compression or distortion.

In [ ]:
from skimage import metrics
from skimage.metrics import structural_similarity
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

gan = cv2.imread('/content/gen_1.png') # generated image
reference = cv2.imread('/content/160_img_.png') #reference image

# Resize images
gan = cv2.resize(gan, (256, 256))
reference = cv2.resize(reference, (256, 256))

# Convert images to grayscale
gan_gray = cv2.cvtColor(gan, cv2.COLOR_BGR2GRAY)
ref_gray = cv2.cvtColor(reference, cv2.COLOR_BGR2GRAY)

# Compute SSIM between two images
(score, diff) = structural_similarity(gan_gray, ref_gray, full=True)
print("Image similarity:", score)

# Convert diff to 8-bit unsigned integer in the range [0,255]
diff = (diff * 255).astype("uint8")

# Threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

mask = np.zeros(gan.shape, dtype='uint8')
filled_after = reference.copy()

for c in contours:
    area = cv2.contourArea(c)
    if area > 40:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(gan, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.rectangle(reference, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.drawContours(mask, [c], 0, (0,255,0), -1)
        cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)

# Display images and differences using cv2_imshow
cv2_imshow(gan)
cv2_imshow(reference)
cv2_imshow(diff)
cv2_imshow(mask)
cv2_imshow(filled_after)


# FID (Fréchet Inception Distance):

FID compares the feature representations of real and generated images using a pre-trained deep learning model. It measures the similarity between the distributions of real and generated images in feature space.
Lower FID values indicate better similarity between real and generated images.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from scipy.linalg import sqrtm
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def calculate_fid(real_images, generated_images):
    # Load pre-trained InceptionV3 model
    inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

    # Preprocess images
    real_images = preprocess_input(real_images)
    generated_images = preprocess_input(generated_images)

    # Get features from Inception model
    real_features = inception_model.predict(real_images)
    generated_features = inception_model.predict(generated_images)

    # Calculate mean and covariance of real and generated features
    mu_real, sigma_real = np.mean(real_features, axis=0), np.cov(real_features, rowvar=False)
    mu_generated, sigma_generated = np.mean(generated_features, axis=0), np.cov(generated_features, rowvar=False)

    # Reshape covariance matrices to 2D
    sigma_real = np.atleast_2d(sigma_real)
    sigma_generated = np.atleast_2d(sigma_generated)

    # Calculate Fréchet distance
    epsilon = 1e-6
    sqrt_sigma_real_generated = sqrtm(sigma_real @ sigma_generated)
    fid = np.linalg.norm(mu_real - mu_generated) + np.trace(sigma_real + sigma_generated - 2 * sqrt_sigma_real_generated)

    return fid, real_features, generated_features

# Visualize t-SNE embeddings
def visualize_tsne(real_features, generated_features):
    all_features = np.concatenate([real_features, generated_features], axis=0)
    all_labels = np.concatenate([np.ones(len(real_features)), np.zeros(len(generated_features))], axis=0)

    # Choose a perplexity value less than the number of samples
    perplexity = min(30, len(all_features) - 1)

    tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42)
    embedded_features = tsne.fit_transform(all_features)

    plt.figure(figsize=(5, 3))
    plt.scatter(embedded_features[all_labels == 1, 0], embedded_features[all_labels == 1, 1], c='blue', label='Real')
    plt.scatter(embedded_features[all_labels == 0, 0], embedded_features[all_labels == 0, 1], c='red', label='Generated')
    plt.title('t-SNE visualization of Inception features')
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.legend()
    plt.show()

# Load and preprocess images
real_image_path = '/content/160_img_.png' # reference image
generated_image_path = '/content/gen_1.png' #generated image

real_image = img_to_array(load_img(real_image_path, target_size=(299, 299)))
generated_image = img_to_array(load_img(generated_image_path, target_size=(299, 299)))

# Convert images to numpy arrays
real_image = np.expand_dims(real_image, axis=0)
generated_image = np.expand_dims(generated_image, axis=0)

# Assume real_image and generated_image are numpy arrays of shape (1, height, width, channels)
fid_score, real_features, generated_features = calculate_fid(real_image, generated_image)
print("Fréchet Inception Distance (FID):", fid_score)

# Visualize t-SNE embeddings
visualize_tsne(real_features, generated_features)

# F1-SCORE
F1-Score is a measure of a test's accuracy that considers both the precision and recall of the test to compute the score. It is the harmonic mean of precision and recall.


In [ ]:
from sklearn.metrics import f1_score
import cv2
import numpy as np

# Load and preprocess images
real_image_path = '/content/160_img_.png' # reference image
generated_image_path = '/content/gen_1.png' # generated image

real_image = cv2.imread(real_image_path)
generated_image = cv2.imread(generated_image_path)

# Resize images to the same dimensions if needed
real_image = cv2.resize(real_image, (generated_image.shape[1], generated_image.shape[0]))

# Compute absolute pixel-wise difference between the generated and real images
difference_image = cv2.absdiff(real_image, generated_image)

# Apply a threshold to binarize the difference image
threshold = 30
_, binary_mask = cv2.threshold(cv2.cvtColor(difference_image, cv2.COLOR_BGR2GRAY), threshold, 255, cv2.THRESH_BINARY)

# Flatten the binary masks for both the generated and real images
flatten_generated_mask = binary_mask.flatten()  # Treat this as predicted labels
flatten_real_mask = np.zeros_like(flatten_generated_mask)  # Treat this as true labels

# Compute F1 score

f1 = f1_score(flatten_real_mask, flatten_generated_mask, pos_label=0)

print("F1 Score based on image difference:", f1)



# SSEQ  (Spatial-Spectral Entropy-based Quality):

It computes the entropy of both spatial and spectral domains and calculates the index based on these values.It can be used to assess the fidelity and similarity of generated images compared to real images.

In [ ]:
import cv2
import numpy as np

def compute_entropy(image):
    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Compute histogram
    hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])

    # Normalize histogram
    hist /= np.sum(hist)

    # Compute entropy
    entropy = -np.sum(hist * np.log2(hist + 1e-10))

    return entropy

def compute_spatial_spectral_entropy(real_image_path, generated_image_path):
    # Load images
    real_image = cv2.imread(real_image_path)
    generated_image = cv2.imread(generated_image_path)

    # Compute entropy for real and generated images
    real_spatial_entropy = compute_entropy(real_image)
    generated_spatial_entropy = compute_entropy(generated_image)

    # Compute entropy for spectral domain
    real_spectral_entropy = compute_entropy(real_image)
    generated_spectral_entropy = compute_entropy(generated_image)

    # Compute SSEQ index
    sseq_index = (real_spatial_entropy / generated_spatial_entropy) * (real_spectral_entropy / generated_spectral_entropy)

    return sseq_index

# Example usage
real_image_path = '/content/160_img_.png'
generated_image_path = '/content/man_gen.png'
sseq_index = compute_spatial_spectral_entropy(real_image_path, generated_image_path)
print("Spatial-Spectral Entropy-based Quality (SSEQ) Index:", sseq_index)


# UIQM (Universal Image Quality Metric):

UIQM is a universal image quality metric that considers various image quality factors, including sharpness, colorfulness, contrast, and exposure.
It is suitable for assessing the overall quality of images in various applications.

In [ ]:
import numpy as np
import cv2

def get_uiqm(image):
    # Convert the image to LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Calculate mean and standard deviation of each channel
    mean_l, std_l = np.mean(lab_image[:,:,0]), np.std(lab_image[:,:,0])
    mean_a, std_a = np.mean(lab_image[:,:,1]), np.std(lab_image[:,:,1])
    mean_b, std_b = np.mean(lab_image[:,:,2]), np.std(lab_image[:,:,2])

    # Calculate contrast
    contrast = np.sqrt(std_l ** 2 + std_a ** 2 + std_b ** 2)

    # Calculate saturation
    saturation = np.sqrt(std_a ** 2 + std_b ** 2)

    # Calculate average intensity
    avg_intensity = np.mean(image)

    # Calculate exposure
    exposure = (np.mean(image) - 128) ** 2

    # Calculate colorfulness
    rg = image[:,:,0] - image[:,:,1]
    yb = 0.5 * (image[:,:,0] + image[:,:,1]) - image[:,:,2]
    colorfulness = np.sqrt(np.mean(rg ** 2 + yb ** 2))

    # Calculate naturalness
    naturalness = 0.486 * contrast + 0.319 * saturation + 0.115 * avg_intensity + 0.031 * exposure + 0.048 * colorfulness

    return naturalness

# Example usage
if __name__ == "__main__":
    # Load your underwater image
    image = cv2.imread('/content/man_gen.png') #generated image

    # Calculate UIQM
    uiqm_score = get_uiqm(image)

    print("UIQM Score:", uiqm_score)
